## Part 4 - Deploy Model

**Connect to Azure ML workspace and get deployed model**

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()
model = workspace.models['cifar-classifier']

**Define score file and environment (inference configuration)**

In [ ]:
%pycat deploy/score.py

In [ ]:
%pycat deploy/env.yml

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='deploy',
                                   runtime='python',
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy as local web service**

In [ ]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import LocalWebservice
from azureml.core import Model

local_service_name = 'local-cifar-classifier'
local_config = LocalWebservice.deploy_configuration(port=3000)

try:
    local_service = Webservice(workspace, name=local_service_name)
    if local_service:
        local_service.delete()
except WebserviceException as e:
    print()

local_service = Model.deploy(workspace, local_service_name, [model], inference_config, local_config)
local_service.wait_for_deployment()

**Test local service**

In [ ]:
from io import BytesIO
import urllib.request
import base64
import io
import requests 
import json
import argparse
from PIL import Image

def imgToBase64(img):
    '''Convert pillow image to base64-encoded image'''
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

def test_service(image_url, scoring_url):
    # Download image and convert to base 64
    with urllib.request.urlopen(image_url) as url:
        test_img = io.BytesIO(url.read())

    base64Img = imgToBase64(Image.open(test_img))
    
    # Get prediciton through endpoint
    input_data = '{\"data\": \"'+ base64Img +'\"}'
    headers = {'Content-Type':'application/json'}
    response = requests.post(scoring_url, input_data, headers=headers)
    return json.loads(response.text)

In [ ]:
scoring_url = local_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

**Deploy as AKS service**

In [ ]:
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute

aks_compute_target = AksCompute(workspace, 'aks-cluster')

aks_service_name = 'aks-cifar-classifier'
aks_config = AksWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, auth_enabled=False)

try:
    aks_service = Webservice(workspace, name=aks_service_name)
    if aks_service:
        aks_service.delete()
except WebserviceException as e:
    print()

aks_service = Model.deploy(workspace, aks_service_name, [model], inference_config, aks_config, aks_compute_target)
aks_service.wait_for_deployment()

**Test AKS service**

In [ ]:
scoring_url = aks_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)